# Inverse Laplace transform evaluation of chain diagrams

In [1]:
using InverseLaplace: Talbot
using Printf

"""
Make the Laplace image of d_n(α, x) w.r.t. x
"""
function make_Ldn(n, α)
    return s -> factorial(big(2n)) / (s^2 * (s + α)^n * (s + 2α)^(n-1))
end

# Multi-precision Laplace transform inversion
# Abate, J. and Valkó, P.P.
# International Journal for Numerical Methods in Engineering, Vol. 60 (Iss. 5-7)  2004  pp 979–993
# https://doi.org/10.1002/nme.995

setprecision(BigFloat, 4000)

for α in [0.0, 0.5, 1.0, 2.0]
    for n in [1, 2, 3, 4, 5, 10, 1000, 2000, 4000]
        Nterms = max(2n, 100) # max() ensures we use enough terms at low n
        d_n = Talbot(make_Ldn(n, α), Nterms)(big(1.0))
        @printf "α = %.3f, n = %5i\t d_n = %.10e\n" α n d_n;
    end
end

[ Info: Precompiling InverseLaplace [e24f45a5-4790-5a38-8b2c-33d770a4e70f]
ERROR: LoadError: ArgumentError: Package LogExpFunctions does not have ChainRulesCore in its dependencies:
- If you have LogExpFunctions checked out for development and have
  added ChainRulesCore as a dependency but haven't updated your primary
  environment's manifest file, try `Pkg.resolve()`.
- Otherwise you may need to report an issue with LogExpFunctions
Stacktrace:
  [1] require(into::Module, mod::Symbol)
    @ Base ./loading.jl:980
  [2] include(mod::Module, _path::String)
    @ Base ./Base.jl:418
  [3] include(x::String)
    @ LogExpFunctions ~/.julia/packages/LogExpFunctions/W8b4e/src/LogExpFunctions.jl:1
  [4] top-level scope
    @ ~/.julia/packages/LogExpFunctions/W8b4e/src/LogExpFunctions.jl:27
  [5] include
    @ ./Base.jl:418 [inlined]
  [6] include_package_for_output(pkg::Base.PkgId, input::String, depot_path::Vector{String}, dl_load_path::Vector{String}, load_path::Vector{String}, concrete_deps:

LoadError: Failed to precompile InverseLaplace [e24f45a5-4790-5a38-8b2c-33d770a4e70f] to /Users/hugstr/.julia/compiled/v1.7/InverseLaplace/jl_NJvyhq.

In [3]:
α = 1
d_n = [ Talbot(make_Ldn(n, α), max(2n, 100))(big(1.0)) for n in 1:16 ]
@show d_n

LoadError: UndefVarError: make_Ldn not defined

In [ ]:
n_max = 128

β = 1
V = 1
ϵ = 1

Ds = evaluate_diagrams_eps_numpy(n_max, ϵ, β)
print(Ds)
import h5py

ns = np.arange(1, n_max+1)

filename = f'data_analytic_chain_integral_beta_{β}_V_{V}_n_max_{n_max}.h5'
print(f'--> Writing: {filename}')
with h5py.File(filename, 'w') as f:
    g = f.create_group("data")    
    g.create_dataset('Ds', data=np.array(Ds, dtype=np.float64))
    g.create_dataset('ns', data=ns)
    g.attrs['beta'] = β
    g.attrs['epsilon'] = ϵ
    g.attrs['V'] = float(V)